# Emoji generation using LSTM

In [127]:
import numpy as np
from emo_utils import *
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Reshape, Lambda
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
np.random.seed(1)

In [128]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [129]:
index = 4
print(X_train[index], label_to_emoji(Y_train[index]))

food is life 🍴


In [121]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

In [86]:
def sentences_to_indices(X, word_to_index, max_len):
 
    m = X.shape[0]                                   
    X_indices = np.zeros((m, max_len))
    for i in range(m):                               
        
        sentence_words = X[i].lower().split()
        j = 0
        
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w] 
            j = j + 1
    
    return X_indices

In [87]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


In [88]:
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')

In [110]:
maxLen = len(max(X_train, key=len).split())

In [90]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)
print(X_train_indices.shape)
X_train_indices.shape

(132, 10)


(132, 10)

In [91]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):

    vocab_len = len(word_to_index) + 1                  
    emb_dim = word_to_vec_map["cucumber"].shape[0]
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)

    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [92]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0].shape)

weights[0][1][3] = (400001, 50)


In [93]:
n_a = 128
n_values = 5

In [94]:
reshapor = Reshape((1, 50))                        # Used in Step 2.B of djmodel(), below
LSTM_cell = LSTM(n_a, return_state = True)         # Used in Step 2.C
densor = Dense(n_values, activation='softmax')     # Used in Step 2.D

In [95]:
global_a0 =np.zeros((X_train.shape[0], n_a))
global_c0 =np.zeros((X_train.shape[0], n_a))

In [96]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index, n_a):

    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    X = embeddings
    print("Shape of embeddings: ", embeddings.shape)
    n_value = embeddings.shape[2]
    Tx = embeddings.shape[1]
    
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    for t in range(Tx-1):
        
        x = Lambda(lambda x: X[:,t,:])(X)
        x = reshapor(x)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
    
    x = Lambda(lambda x: X[:,t,:])(X)
    x = reshapor(x)
    a, _, c = LSTM_cell(x, initial_state=[a, c])
    out = densor(a)
    
    model = Model(inputs = [sentence_indices, a0, c0], outputs = out)
    
    return model

In [97]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index, n_a)

Shape of embeddings:  (?, 10, 50)


In [98]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [99]:
m = X_train.shape[0]
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

In [100]:
model.fit([X_train_indices, a0, c0], Y_train_oh, epochs=100)

Epoch 1/100
132/132 [==============================] - 20s 152ms/step - loss: 1.6650 - acc: 0.3409
Epoch 2/100
132/132 [==============================] - 0s 2ms/step - loss: 1.2873 - acc: 0.5227
Epoch 3/100
132/132 [==============================] - 0s 3ms/step - loss: 1.2015 - acc: 0.5152
Epoch 4/100
132/132 [==============================] - 0s 2ms/step - loss: 0.8470 - acc: 0.6667
Epoch 5/100
132/132 [==============================] - 0s 2ms/step - loss: 0.8715 - acc: 0.6591
Epoch 6/100
132/132 [==============================] - 0s 2ms/step - loss: 0.7226 - acc: 0.7121
Epoch 7/100
132/132 [==============================] - 0s 2ms/step - loss: 0.6642 - acc: 0.7273
Epoch 8/100
132/132 [==============================] - 0s 2ms/step - loss: 0.6558 - acc: 0.7500
Epoch 9/100
132/132 [==============================] - 0s 2ms/step - loss: 0.5753 - acc: 0.7955
Epoch 10/100
132/132 [==============================] - 0s 1ms/step - loss: 0.5119 - acc: 0.8106
Epoch 11/100
132/132 [==============

132/132 [==============================] - 0s 720us/step - loss: 3.8744e-04 - acc: 1.0000
Epoch 83/100
132/132 [==============================] - 0s 735us/step - loss: 3.8227e-04 - acc: 1.0000
Epoch 84/100
132/132 [==============================] - 0s 773us/step - loss: 3.7597e-04 - acc: 1.0000
Epoch 85/100
132/132 [==============================] - 0s 758us/step - loss: 3.6908e-04 - acc: 1.0000
Epoch 86/100
132/132 [==============================] - 0s 720us/step - loss: 3.6346e-04 - acc: 1.0000
Epoch 87/100
132/132 [==============================] - 0s 728us/step - loss: 3.5845e-04 - acc: 1.0000
Epoch 88/100
132/132 [==============================] - 0s 735us/step - loss: 3.5293e-04 - acc: 1.0000
Epoch 89/100
132/132 [==============================] - 0s 811us/step - loss: 3.4833e-04 - acc: 1.0000
Epoch 90/100
132/132 [==============================] - 0s 826us/step - loss: 3.4393e-04 - acc: 1.0000
Epoch 91/100
132/132 [==============================] - 0s 788us/step - loss: 3.3913e-

In [101]:
def emoji_inference_model(LSTM_cell, densor, input_shape, word_to_vec_map, word_to_index, n_a):

    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    X = embeddings
    print("Shape of embeddings: ", embeddings.shape)
    n_value = embeddings.shape[2]
    Tx = embeddings.shape[1]
    
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    for t in range(Tx-1):
        
        x = Lambda(lambda x: X[:,t,:])(X)
        x = reshapor(x)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
    
    x = Lambda(lambda x: X[:,t,:])(X)
    x = reshapor(x)
    a, _, c = LSTM_cell(x, initial_state=[a, c])
    out = densor(a)
    
    model = Model(inputs = [sentence_indices, a0, c0], outputs = out)
    
    return model

In [102]:
inference_model = emoji_inference_model(LSTM_cell, densor, (maxLen,), word_to_vec_map, word_to_index, n_a)

Shape of embeddings:  (?, 10, 50)


In [103]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)

In [104]:
m = X_test.shape[0]
a_initializer = np.zeros((m, n_a))
c_initializer = np.zeros((m, n_a))

In [108]:
def predict_and_evaluate(inference_model, x_initializer, a_initializer, c_initializer, Y_test_oh):

    eval_ = model.evaluate([X_test_indices, a_initializer, c_initializer], Y_test_oh)
    
    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
    
    for i in range(len(X_test)):
        num = np.argmax(pred[i])
        if(num != Y_test[i]):
            print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

    return eval_[0], eval_[1]

In [109]:
loss, acc = predict_and_evaluate(inference_model, X_test_indices, a_initializer, c_initializer, Y_test_oh)
print("Accuracy: ", acc)

56/56 [==============================] - 5s 82ms/step
Expected emoji:😄 prediction: he got a very nice raise	❤️
Expected emoji:😄 prediction: she got me a nice present	❤️
Expected emoji:😞 prediction: This girl is messing with me	❤️
Expected emoji:😄 prediction: you brighten my day	❤️
Expected emoji:😞 prediction: she is a bully	❤️
Expected emoji:😞 prediction: My life is so boring	❤️
Expected emoji:😄 prediction: will you be my valentine	❤️
Expected emoji:⚾ prediction: he can pitch really well	❤️
Expected emoji:❤️ prediction: family is all I have	😞
Accuracy:  0.839285705770765
